In [1]:
import sys
sys.path.append('/Users/andreas/fun/LED/pyTibber')
from pyTibber import tibber


In [3]:
async def main():
    # Replace with your actual Tibber API token
    TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
    try:
        # Initialize Tibber connection
        tibber_connection = tibber.Tibber(access_token=TOKEN)
        await tibber_connection.update_info()

        # Retrieve the websocketSubscriptionUrl
        if tibber_connection.realtime.sub_endpoint:
            print("WebSocket Subscription URL:", tibber_connection.realtime.sub_endpoint)

        # Get hourly PriceRating and PriceLevel for each active home
        for home_id in tibber_connection.get_home_ids(only_active=True):
            home = tibber_connection.get_home(home_id)
            if home is not None:
                await home.update_price_info()
                if home.current_price_info:
                    price_info = home.current_price_info
                    print(f"Home ID: {home_id}")
                    print(f"Hourly PriceRating: {price_info['total']}")
                    print(f"PriceLevel: {price_info['priceLevel']}")
                    
        # Closing the connection
        await tibber_connection.close_connection()
    except Exception as e:
        print(f"An error occurred: {e}")

    if __name__ == "__main__":
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())
        


In [13]:
import asyncio
#from pyTibber import Tibber
async def main():
    TOKEN = "U4L8yS_OHsfgKndAhNQZ8K-JYElbNUagYvToCF3ZPVE"
    USER_AGENT = "LED_client"
    tibber_connection = tibber.Tibber(access_token=TOKEN,user_agent=USER_AGENT)
    await tibber_connection.update_info()

    # GraphQL query to fetch home information
    query = '''
    {
      viewer {
        homes {
          id
          address {
            address1
            city
          }
          owner {
            name
          }
          currentSubscription {
            priceInfo {
              current {
                total
                level
              }
            }
          }
        }
      }
    }
    '''

    data = await tibber_connection.execute(query)
    if data:
        homes = data['viewer']['homes']
        for home in homes:
            print(f"Home ID: {home['id']}")
            print(f"Address: {home['address']['address1']}, {home['address']['city']}")
            print(f"Owner's Name: {home['owner']['name']}")

            # Check if currentSubscription and priceInfo are not None
            if home.get('currentSubscription') and home['currentSubscription'].get('priceInfo'):
                current_price_info = home['currentSubscription']['priceInfo'].get('current')
                if current_price_info:
                    print(f"Current Power Price: {current_price_info['total']}")
                    print(f"Price Level: {current_price_info['level']}")
            print()

    await tibber_connection.close_connection()
await main()


Home ID: f65b179b-f74f-492c-9ea8-db1e1461856a
Address: Helmers vei 10, LAKSEVÅG
Owner's Name: Erlend Thue

Home ID: 975996b6-e7ca-4fbf-9f72-61e2df95bc0c
Address: Løbergsalleen 16, BERGEN
Owner's Name: Erlend Thue
Current Power Price: 1.2303
Price Level: CHEAP



In [7]:

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


RuntimeError: This event loop is already running